In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [6]:
tf.__version__

'2.11.0'

# 데이터 전처리

### 훈련 세트

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255, # 각 픽셀 값이 0~255이기 때문에 255로 나눠서 스케일링한다.
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True) # 과적합 예방
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), 
                                                 batch_size = 32, # 크기 조절
                                                 class_mode = 'binary') # 개 or 고양이 이진수

Found 8000 images belonging to 2 classes.


### 테스트 세트

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


# CNN 구현

In [9]:
cnn = tf.keras.models.Sequential()

합성곱 구현

In [10]:
# 3*3프레임, 활성 Relu함수, 컬러이기 때문에 3 (흑백은 1)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

풀링

In [11]:
# 2*2 프레임, 필수요소로 2픽셀마다 바꾼다.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

합성곱 추가하기

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Flattening

In [13]:
cnn.add(tf.keras.layers.Flatten())

완전 연결

In [14]:
# 뉴런 수 128개 선택, 활성함수 Relu
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

출력 계층

In [16]:
# 이진 수이기 때문에 뉴런 수는 1개, 출력층에서는 Sigmoid (다중 분류에서는 Softmax함수)
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# 훈련하기

In [17]:
# 이진 분류인것을 잊지 말자.
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
# 사진의 수는 8000개, 배치 사이즈가 32이기 때문에 epoch마다 250단계로 되어 있음.
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 100s 392ms/step - loss: 0.6853 - accuracy: 0.5661 - val_loss: 0.6729 - val_accuracy: 0.6235
Epoch 2/25
250/250 [==============================] - 83s 330ms/step - loss: 0.6558 - accuracy: 0.6482 - val_loss: 0.6429 - val_accuracy: 0.6770
Epoch 3/25
250/250 [==============================] - 88s 351ms/step - loss: 0.6455 - accuracy: 0.6616 - val_loss: 0.6207 - val_accuracy: 0.7105
Epoch 4/25
250/250 [==============================] - 57s 227ms/step - loss: 0.6280 - accuracy: 0.6848 - val_loss: 0.6454 - val_accuracy: 0.6345
Epoch 5/25
250/250 [==============================] - 56s 224ms/step - loss: 0.6210 - accuracy: 0.6894 - val_loss: 0.6179 - val_accuracy: 0.6905
Epoch 6/25
250/250 [==============================] - 57s 226ms/step - loss: 0.6087 - accuracy: 0.7054 - val_loss: 0.6129 - val_accuracy: 0.6895
Epoch 7/25
250/250 [==============================] - 56s 223ms/step - loss: 0.5971 - accuracy: 0.7149 - val_loss: 0.5895 - val_a

훈련한 결과로 실험을 해보자

In [23]:
import numpy as np
from keras.preprocessing import image
import keras.utils as image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) # 사이즈는 같게
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # 이미지를 추가하고 싶을 때 
result = cnn.predict(test_image)

# 강아지를 1, 고양이를 0으로 출력
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 23ms/step


In [24]:
print(prediction)

cat
